# Steps to publish DC Crime data (2008-2017) as a feature service

__Requirements:__
> 1. ArcGIS pro (v 1.4 or higher)
> 2. 'arcgis' package installed with ArcGIS Pro (follow [instructions](https://developers.arcgis.com/python/guide/install-and-set-up/) under Step 2 (Install using ArcGIS Pro)

Once installed, locate the proenv.bat within '\ArcGIS\Pro\bin\python\scripts\', launch terminal/command prompt and execute the following command

__`path_to_this_file\proenv.bat`__

Then launch a jupyter notebook instance. This ensures that Python and the ArcGIS API for Python that is integrated with ArcGIS Pro is used to implement the rest of this notebook that follows.

In [1]:
import arcpy
import pandas as pd

In [2]:
from arcgis.features import FeatureLayer
from arcgis.gis import *
gis = GIS("http://dcdev.maps.arcgis.com/", "username", "password")

In [3]:
#Server location for all DC Crime data
url1 = 'https://maps2.dcgis.dc.gov/dcgis/rest/services/FEEDS/MPD/MapServer/'

In [4]:
#Looping through to combine data of all years
all_crime = pd.DataFrame()
years = ['32', '33', '34', '35', '11', '10', '9', '27', '26', '38']
for i in years:
    url = url1 + i
    try:
        data_layer = FeatureLayer(url)
        #Extracts all data for specific year
        year_features = data_layer.query()
        #Converts it to a pandas dataframe
        year_data = year_features.df
        #Add the new month below the existing data in all_crime
        all_crime = pd.concat([all_crime, year_data])
    except:
        print(i)

In [5]:
all_crime.shape

(336850, 25)

In [6]:
all_crime.head()

,ANC,BID,BLOCK,BLOCK_GROUP,CCN,CENSUS_TRACT,DISTRICT,END_DATE,LATITUDE,LONGITUDE,...,REPORT_DAT,SHIFT,START_DATE,VOTING_PRECINCT,WARD,XBLOCK,XCOORD,YBLOCK,YCOORD,SHAPE
0,7D,None,1900 - 2199 BLOCK OF EAST CAPITOL STREET,006804 1,08114924,006804,1,1.218755e+12,38.890104,-76.975437,...,1218754800000,EVENING,1.218752e+12,Precinct 80,7,402131.000000,402347.350,135807.00000,135781.36,"{'x': 402131, 'y': 135807}"
1,8E,None,900 - 999 BLOCK OF BARNABY STREET SE,009801 1,08114963,009801,7,1.218672e+12,38.832904,-76.990195,...,1218761400000,MIDNIGHT,1.218756e+12,Precinct 121,8,400851.299167,400851.299,129457.20047,129457.20,"{'x': 400851.2991999984, 'y': 129457.20050000027}"
2,2B,None,1620 - 1699 BLOCK OF 15TH STREET NW,005201 2,08114980,005201,3,1.218755e+12,38.912235,-77.034556,...,1218763320000,MIDNIGHT,1.218677e+12,Precinct 16,2,397003.000000,396978.830,138264.00000,138283.08,"{'x': 397003, 'y': 138264}"
3,7F,None,3826 - 3999 BLOCK OF BLAINE STREET NE,009603 2,08114985,009603,6,1.218760e+12,38.891888,-76.948809,...,1218771900000,MIDNIGHT,1.218760e+12,Precinct 102,7,404441.000000,404452.740,136006.00000,136027.68,"{'x': 404441, 'y': 136006}"
4,4C,None,1300 - 1399 BLOCK OF GALLATIN STREET NW,002002 3,08114995,002002,4,1.218766e+12,38.951872,-77.031564,...,1218768300000,MIDNIGHT,1.218766e+12,Precinct 54,4,397264.000000,397404.160,142664.00000,142640.72,"{'x': 397264, 'y': 142664}"


In [7]:
#Renaming columns to suit ArcGIS online requirements
all_crime.rename(columns={'XCOORD': 'x', 'YCOORD': 'y'}, inplace=True)

In [8]:
#Indexing the entire dataframe
all_crime.reset_index(inplace=True)

In [9]:
#Displaying data types of every column
all_crime.dtypes

index                     int64
ANC                      object
BID                      object
BLOCK                    object
BLOCK_GROUP              object
CCN                      object
CENSUS_TRACT             object
DISTRICT                 object
END_DATE                float64
LATITUDE                float64
LONGITUDE               float64
METHOD                   object
NEIGHBORHOOD_CLUSTER     object
OBJECTID                  int64
OFFENSE                  object
PSA                      object
REPORT_DAT                int64
SHIFT                    object
START_DATE              float64
VOTING_PRECINCT          object
WARD                     object
XBLOCK                  float64
x                       float64
YBLOCK                  float64
y                       float64
SHAPE                    object
dtype: object

Here, the column type of `REPORT_DAT` will have to be changed to String type, else the API doesn't allow publishing the data.

In [11]:
all_crime['REPORT_DAT'] = all_crime['REPORT_DAT'].astype(str)

In [12]:
all_crime.head()

,index,ANC,BID,BLOCK,BLOCK_GROUP,CCN,CENSUS_TRACT,DISTRICT,END_DATE,LATITUDE,...,REPORT_DAT,SHIFT,START_DATE,VOTING_PRECINCT,WARD,XBLOCK,x,YBLOCK,y,SHAPE
0,0,7D,None,1900 - 2199 BLOCK OF EAST CAPITOL STREET,006804 1,08114924,006804,1,1.218755e+12,38.890104,...,1218754800000,EVENING,1.218752e+12,Precinct 80,7,402131.000000,402347.350,135807.00000,135781.36,"{'x': 402131, 'y': 135807}"
1,1,8E,None,900 - 999 BLOCK OF BARNABY STREET SE,009801 1,08114963,009801,7,1.218672e+12,38.832904,...,1218761400000,MIDNIGHT,1.218756e+12,Precinct 121,8,400851.299167,400851.299,129457.20047,129457.20,"{'x': 400851.2991999984, 'y': 129457.20050000027}"
2,2,2B,None,1620 - 1699 BLOCK OF 15TH STREET NW,005201 2,08114980,005201,3,1.218755e+12,38.912235,...,1218763320000,MIDNIGHT,1.218677e+12,Precinct 16,2,397003.000000,396978.830,138264.00000,138283.08,"{'x': 397003, 'y': 138264}"
3,3,7F,None,3826 - 3999 BLOCK OF BLAINE STREET NE,009603 2,08114985,009603,6,1.218760e+12,38.891888,...,1218771900000,MIDNIGHT,1.218760e+12,Precinct 102,7,404441.000000,404452.740,136006.00000,136027.68,"{'x': 404441, 'y': 136006}"
4,4,4C,None,1300 - 1399 BLOCK OF GALLATIN STREET NW,002002 3,08114995,002002,4,1.218766e+12,38.951872,...,1218768300000,MIDNIGHT,1.218766e+12,Precinct 54,4,397264.000000,397404.160,142664.00000,142640.72,"{'x': 397264, 'y': 142664}"


In [13]:
#Publishing the data
crime_layer = gis.content.import_data(all_crime, title='AllCrime_Aug31')

In [14]:
#Verifying it
search_result = gis.content.search("AllCrime_Aug31")
search_result[0]

<Item title:"AllCrime_Aug31" type:Feature Layer Collection owner:mmajumdar_dcdev>